# Handout 14

# **Sinais II: Recebimento e Concorrência**

## **1. Revisão de Sinais: `wait` e `kill`**

Os sinais são uma forma de notificação assíncrona que permite ao sistema operacional informar processos sobre eventos específicos, como exceções de hardware ou mudanças no sistema. 

---

## **2. Recebendo um Sinal**

Quando um sinal é enviado a um processo, o kernel força o processo destinatário a reagir de alguma forma. As opções são:

- **Ignorar o sinal**: O processo não realiza nenhuma ação.
- **Terminar o processo**: Pode gerar um *core dump*.
- **Capturar o sinal**: Executa uma função específica chamada de *signal handler*.

### **Captura de Sinais**

Um sinal funciona como uma exceção de hardware e a execução segue os seguintes passos:
1. O sinal é recebido pelo processo.
2. O controle é transferido para o *handler* do sinal.
3. O *handler* do sinal é executado.
4. O controle retorna para a próxima instrução.

---

## **3. Função `sigaction` para Captura de Sinais**

A função `sigaction` é usada para definir como o processo reage a sinais:

```c
#include <signal.h>

int sigaction(int signum, const struct sigaction *act, struct sigaction *oldact);



- signum: Número do sinal.
- act: Nova ação a ser executada.
- oldact: Salva a ação anterior.

### Estrutura sigaction

In [ ]:
struct sigaction {
    void     (*sa_handler)(int);
    void     (*sa_sigaction)(int, siginfo_t *, void *);
    sigset_t   sa_mask;
    int        sa_flags;
    void     (*sa_restorer)(void);
};

- sa_handler: Função para manipular o sinal.
- sa_mask: Sinais a serem bloqueados durante a execução do handler.
- sa_flags: Modifica o comportamento do sinal.

## **4. Exemplo Prático: Capturando Sinais com `sigaction`**

In [ ]:
#include <stdio.h>
#include <signal.h>
#include <unistd.h>

void handle_signal(int sig) {
    printf("Sinal %d recebido\n", sig);
}

int main() {
    struct sigaction sa;
    sa.sa_handler = handle_signal;
    sa.sa_flags = 0; // Opções padrão
    sigemptyset(&sa.sa_mask); // Nenhum sinal bloqueado

    sigaction(SIGINT, &sa, NULL); // Captura SIGINT (Ctrl+C)

    while (1) {
        pause(); // Aguarda sinais
    }
    return 0;
}


In [ ]:
// Usando como exemplo o código acima,
// modifique o arquivo sinal1.c para que o programa só termine após apertar Ctrl+C três vezes. 
//Você pode usar exit para sair na terceira vez. 
//Não se esqueça de consultar man sigaction para verificar quais includes devem ser usados.

#include <stdio.h>
#include <unistd.h>
#include <signal.h>
#include <stdlib.h>
#include <sys/types.h>

int contador = 0;

void sig_handler(int num) {
    contador++;
    printf("Chamou Ctrl+C\n");
    if (contador == 3) {
        exit(0);
    }
}

int main() {

    struct sigaction handler;

    handler.sa_handler = sig_handler;
    handler.sa_flags = 0;
    sigemptyset(&handler.sa_mask);

    sigaction(SIGINT, &handler, NULL);

    printf("Meu pid: %d\n", getpid());

    while(1) {
        sleep(1);
    }
    return 0;
}

## **5. Concorrência e Sinais**

### **Problemas de Concorrência**

Quando dois *handlers* tentam acessar o mesmo recurso (como uma variável global ou a função `printf`), podem ocorrer problemas de concorrência.  
**Exemplo de problema**:
- Um *handler* atualiza uma variável enquanto outro tenta lê-la.
- A saída pode ser imprevisível se ambos os *handlers* executarem simultaneamente.

**Cenário típico**:
- Dois *handlers* lidando com sinais diferentes podem ser acionados ao mesmo tempo.
- É crucial garantir que um *handler* não interfira no estado ou no fluxo de outro.

---

Exemplo:

In [ ]:
volatile int contador = 0;

void handler1(int sig) {
    contador++;
}

void handler2(int sig) {
    printf("Contador: %d\n", contador);
}

//Se handler1 e handler2 forem executados simultaneamente, podem ocorrer inconsistências.

## **6. Bloqueio de Sinais**

### **Bloquear Sinais Temporariamente**

Sinais podem ser bloqueados para evitar conflitos enquanto um *handler* é executado.  
Isso é realizado utilizando a máscara de sinais (`sa_mask`) na estrutura `sigaction`.

### **Como Funciona o Bloqueio de Sinais**:
- Enquanto um sinal está bloqueado, ele permanece pendente.
- O sinal bloqueado é entregue ao processo assim que for desbloqueado, garantindo que o *handler* atual finalize sua execução sem interrupções.

**Importância**:
- Previne condições de corrida.
- Assegura que o fluxo do programa não seja interrompido de forma imprevisível.

Exemplo:

In [ ]:
#include <stdio.h>
#include <signal.h>
#include <unistd.h>

void handle_signal(int sig) {
    printf("Sinal %d recebido\n", sig);
}

int main() {
    struct sigaction sa;
    sa.sa_handler = handle_signal;
    sigemptyset(&sa.sa_mask); 
    sigaddset(&sa.sa_mask, SIGTERM); // Bloqueia SIGTERM durante o handler
    sa.sa_flags = 0;

    sigaction(SIGINT, &sa, NULL); // Captura SIGINT

    while (1) {
        pause(); // Aguarda sinais
    }
    return 0;
}

//Resultado: Enquanto o sinal SIGINT é tratado, o sinal SIGTERM fica bloqueado e só será entregue após o término do handler.